# Building a scan programatically

In this notebook we'll build up a scan definition from first principles, against a local model trained within the
notebook.  We will then run that scan and save its results.  Finally we will extract the scan defintion as YAML, which could be used to run the same scan (potentially on revised models or datasets) via the Certifai stand-alone scanner.

In [ ]:
import pandas as pd
import matplotlib as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from copy import copy

from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiPredictorWrapper, CertifaiModel, CertifaiModelMetric,
                                      CertifaiDataset, CertifaiGroupingFeature, CertifaiDatasetSource,
                                      CertifaiPredictionTask, CertifaiTaskOutcomes, CertifaiOutcomeValue)
from certifai.scanner.report_utils import scores, construct_scores_dataframe

In [ ]:
# special import - 
# for multiprocessing to work in a Notebook,  pickled classes must be in a separate package or notebook
# hence, the encoder class has to be somewhere other than the current notebook
# from ipynb.fs.defs.cat_encoder import CatEncoder # <- doesn't work on Azure Notebooks
# %run cat_encoder.py # <- doesn't work because code doesn't remain external

# Azure Notebooks workaround - 
import os
import sys
sys.path.append("../utils")
from cat_encoder import CatEncoder
from sklearn_soft_wrapper import SkLearnSoftWrapper

In [ ]:
# Example will use a simple logistic classifier on the German Credit dataset
base_path = '..'
all_data_file = f"{base_path}/datasets/german_credit_eval.csv"

df = pd.read_csv(all_data_file)

cat_columns = [
    'checkingstatus',
    'history',
    'purpose',
    'savings',
    'employ',
    'status',
    'others',
    'property',
    'age',
    'otherplans',
    'housing',
    'job',
    'telephone',
    'foreign'
    ]

label_column = 'outcome'

# Separate outcome
y = df[label_column]
X = df.drop(label_column, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Note - to support python multi-processing in the context of a notebook the encoder MUST
# be in a separate file, which is why `CatEncoder` is defined outside of this notebook
encoder = CatEncoder(cat_columns, X)

def build_model(data, name, model_family, test=None):
    if test is None:
        test = data
        
    if model_family == 'SVM':
        parameters = {'probability': [True], 'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, .5, 1, 2, 4, 10], 'gamma':['auto']}
        m = svm.SVC()
    elif model_family == 'logistic':
        parameters = {'C': (0.5, 1.0, 2.0), 'solver': ['lbfgs'], 'max_iter': [1000]}
        m = LogisticRegression()
    model = GridSearchCV(m, parameters, cv=3)
    model.fit(data[0], data[1])

    # Assess on the test data
    accuracy = model.score(test[0], test[1].values)
    print(f"Model '{name}' accuracy is {accuracy}")
    return model

svm_model = build_model((encoder(X_train.to_numpy()), y_train),
                        'Support Vector Machine',
                        'SVM',
                        test=(encoder(X_test.to_numpy()), y_test))

logistic_model = build_model((encoder(X_train.to_numpy()), y_train),
                        'Logistic classifier',
                        'logistic',
                        test=(encoder(X_test.to_numpy()), y_test))

In [ ]:
# Wrap the models using their soft outputs for Certifai

def _wrap_sklearn_classifier(classifier, encoder=None) -> CertifaiPredictorWrapper:
    return CertifaiPredictorWrapper(SkLearnSoftWrapper(classifier),
                                    soft_predictions=True,
                                    encoder=encoder,
                                    label_ordering=classifier.classes_)

svm_model_proxy = _wrap_sklearn_classifier(svm_model, encoder=encoder)
logistic_model_proxy = _wrap_sklearn_classifier(logistic_model, encoder=encoder)

In [ ]:
# Create the scan object from scratch using the ScanBuilder class

# First define the possible prediction outcomes
task = CertifaiPredictionTask(CertifaiTaskOutcomes.classification(
    [
        CertifaiOutcomeValue(1, name='Loan granted', favorable=True),
        CertifaiOutcomeValue(2, name='Loan denied')
    ]),
    prediction_description='Determine whether a loan should be granted')

scan = CertifaiScanBuilder.create('test_user_case',
                                  prediction_task=task)

# Add our local models
first_model = CertifaiModel('SVM',
                            local_predictor=svm_model_proxy)
scan.add_model(first_model)
second_model = CertifaiModel('logistic',
                            local_predictor=logistic_model_proxy)
scan.add_model(second_model)

# Add the eval dataset
eval_dataset = CertifaiDataset('evaluation',
                               CertifaiDatasetSource.csv(all_data_file))
scan.add_dataset(eval_dataset)

# Setup an evaluation for fairness on the above dataset using the model
# We'll look at disparity between groups defined by marital status and age
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('age'))
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('status'))
scan.add_evaluation_type('fairness')
scan.evaluation_dataset_id = 'evaluation'

# Because the dataset contains a ground truth outcome column which the model does not
# expect to receive as input we need to state that in the dataset schema (since it cannot
# be inferred from the CSV)
scan.dataset_schema.outcome_feature_name = 'outcome'

# Run the scan.
# By default this will write the results into individual report files (one per model and evaluation
# type) in the 'reports' directory relative to the Jupyter root.  This may be disabled by specifying
# `write_reports=False` as below
# The result is a dictionary of dictionaries of reports.  The top level dict key is the evaluation type
# and the second level key is model id.
# Reports saved as JSON (which `write_reports=True` will do) may be visualized in the console app
result = scan.run(write_reports=False)

In [ ]:
# The result is a dictionary keyed on analysis, containing reports keyed on model id
# The console app is the recommended way to view these, by saving the results to file
# (see previous cell), but programmatic analysis of the result here is also possible
print(result['fairness']['SVM'].keys())
df = construct_scores_dataframe(scores('fairness', result), include_confidence=False)
display(df)

In [ ]:
# Many scores also come with 95% confidence bounds, which we omitted above for the sake of brevity, but
# we can include those also.  In the example here we include the confidence bounds but only display the scores
# to a reduced level of detail to keep a smallish table for display purposes
df = construct_scores_dataframe(scores('fairness', result, max_depth=1))

display(df)

print("\n")

# Let's chart the fairness measure by feature for each model together with its confidence bounds for
# easier visual comparison:
%matplotlib inline
import matplotlib.pyplot as plt

features = ['Feature (status)', 'Feature (age)']
feature_scores = df[features]
feature_lower_bounds = df[[f + ' lower bound' for f in features]]
feature_upper_bounds = df[[f + ' upper bound' for f in features]]

fig, ax = plt.subplots(figsize=[12,4])
ax.set_title('Feature fairness by model', fontsize=20)
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:pink']
width = 0.4

ax.set_xticks(np.arange(len(features))+width)
ax.set_xticklabels(features)

for idx in range(len(df)):
    central_values = list(feature_scores.iloc[idx])
    lower_bounds = list(feature_lower_bounds.iloc[idx])
    upper_bounds = list(feature_upper_bounds.iloc[idx])
    lower_errors = [central_values[i] - lower_bounds[i] for i in range(len(central_values))]
    upper_errors = [upper_bounds[i] - central_values[i] for i in range(len(central_values))]

    ax.bar([width/2+idx*width+f_idx for f_idx in range(len(features))],
            central_values,
            width,
            yerr=[lower_errors, upper_errors],
            color=colors[idx],
            label=df.index[idx],
            capsize=10)

ax.legend()
plt.show()

In [ ]:
# We can also persist the template we built and reuse it later as illustrated here
import tempfile
import yaml

temp_file = tempfile.mktemp()
with open(temp_file, "w") as f:
    scan.save(f)
    print(f"Saved template to: {temp_file}")

In [ ]:
reloaded = CertifaiScanBuilder.from_file(temp_file)

# The persisted scan won't contain the model itself, so we'll need to reconnect that
for model in reloaded.models:
    if model.id == 'logistic':
        model.local_predictor=logistic_model_proxy
    elif model.id == 'SVM':
        model.local_predictor=svm_model_proxy

In [ ]:
result = reloaded.run(write_reports=False)

In [ ]:
construct_scores_dataframe(scores('fairness', result), include_confidence=False)

In [ ]:
# We can also process dataframes we have locally already loaded in Pandas rather than
# requiring them to initially be saved off as CSV/JSON files
df2 = pd.read_csv(all_data_file)

# Add this already loaded data as a new dataset and switch to evaluate it
loaded_dataset = CertifaiDataset('loaded', CertifaiDatasetSource.dataframe(df2))
reloaded.add_dataset(loaded_dataset)
reloaded.evaluation_dataset_id = 'loaded'
reloaded.output_path = '../local_reports'

result = reloaded.run()

In [ ]:
construct_scores_dataframe(scores('fairness', result), include_confidence=False)

In [ ]:
# Having developed a scan definition we'll eventually want to be able to export it for use in
# scans of a deployed model.  The main differences then will be:
#  * The model won't be local to the scanner, but will be accessed via the network
#  * The dataset won't be in a local DataFrame but will always be read from (possibly network attached) storage
# This means that the scan template used by the scanner will need dataset sourcing details filling in, and model
# endpoint address specifying.  This can either be done by a post-hoc hand edit of the produced YAML (the file we
# saved off earlier to illustrate persisting the template), or we can add those details programatically prior to
# exporting as illustrated below
exportable_scan = CertifaiScanBuilder.from_file(temp_file)  # reload the scan we've been using in the notebook
exportable_scan.models[0].predict_endpoint = 'http://mymodel/predict'
exportable_scan.datasets[0].source = CertifaiDatasetSource.csv('somefile.csv')

# Export the updated scan as YAML for use by the scanner (we won't actually save it here - just print it)
scan_yaml = exportable_scan.extract_yaml() # could use the method `save` to write direct to a file
print(scan_yaml)